# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import scipy

from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Read updated *Nikos* data

In [2]:
xls = pd.ExcelFile('data/edited_nikos_data.xlsx')
xls.sheet_names

['ANAMET',
 'Anamet data',
 'ISRI',
 'Isri data',
 'UNS data',
 'Elements data',
 'Material data',
 'Process']

In [3]:
uns_data = pd.read_excel(xls, 'UNS data')

uns_data.rename(columns = {'sc:isComposedOf':'elements id'}, inplace = True)

elements_data = pd.read_excel(xls, 'Elements data')

uns_data = pd.merge(uns_data, elements_data, how='inner', on = 'elements id')
isri_data = pd.read_excel(xls, 'Isri data')
anamet_data = pd.read_excel(xls, 'Anamet data')
material_data = pd.read_excel(xls, 'Material data')
elements_data = pd.read_excel(xls, 'Elements data')

# Raw data preprocessing

In [4]:
isri_data.head(2)

,isri id,sc:hasCategory,sc:hasForm,sc:isFreeFrom,sc:contains,sc:hasISRICode,sc:hasSpecification,pc:hasName,sc:isComposedOf,sc:isProcessedBy
0,sc:ISRI_0,Nonferrous Scrap - Red Metals,Wire,NaN,Copper,Berry,"Bare, Uncoated, Unalloyed",No. 1 Copper Wire,sc:Material_249,sc:Process_257
1,sc:ISRI_1,Nonferrous Scrap - Red Metals,Wire and Cable,"Brittle Burnt Wire, Copper Tubing",Copper,Barley,"Clean, Untinned, Uncoated, Unalloyed",No. 1 Copper Wire,sc:Material_249,sc:Process_257


In [5]:
isri_data.shape

(244, 10)

In [6]:
isri_data.isna().sum()

isri id                  0
sc:hasCategory           0
sc:hasForm              43
sc:isFreeFrom           87
sc:contains             68
sc:hasISRICode           0
sc:hasSpecification     12
pc:hasName               7
sc:isComposedOf        109
sc:isProcessedBy       194
dtype: int64

#### Drop unusefull columns

In [7]:
df = isri_data.copy()
df.drop(columns=['isri id', 'sc:hasISRICode', 'sc:isProcessedBy'], inplace=True)
df.shape

(244, 7)

In [8]:
import texthero as hero
from texthero import preprocessing

custom_pipeline = [preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_stopwords]

for col in df.drop('sc:isComposedOf',axis=1).columns:
    df[col] = hero.clean(df[col])

df.head()


,sc:hasCategory,sc:hasForm,sc:isFreeFrom,sc:contains,sc:hasSpecification,pc:hasName,sc:isComposedOf
0,nonferrous scrap red metals,wire,,copper,bare uncoated unalloyed,copper wire,sc:Material_249
1,nonferrous scrap red metals,wire cable,brittle burnt wire copper tubing,copper,clean untinned uncoated unalloyed,copper wire,sc:Material_249
2,nonferrous scrap red metals,miscellaneous wire,excessively leaded tinned soldered copper wire...,copper,unalloyed,copper wire,sc:Material_249
3,nonferrous scrap red metals,clippings punchings bus bars commutator segmen...,,copper,clean unalloyed uncoated,heavy copper solids tubing,sc:Material_249
4,nonferrous scrap red metals,miscellaneous scrap,excessively leaded tinned soldered copper scra...,copper,unalloyed,copper solids tubing,sc:Material_249


In [9]:
df.isna().sum()

sc:hasCategory           0
sc:hasForm               0
sc:isFreeFrom            0
sc:contains              0
sc:hasSpecification      0
pc:hasName               0
sc:isComposedOf        109
dtype: int64

## Predict `sc:isComposedOf`

#### TfIdfVectorizer

In [10]:
df = df[df['sc:isComposedOf'].notna()]
df.shape

(135, 7)

In [13]:
df['sc:hasCategory'].value_counts()

nonferrous scrap red metals                                                                  39
ferrous scrap                                                                                26
nonferrous scrap aluminum                                                                    21
nonferrous scrap nickel stainless hi temp                                                    17
nonferrous scrap zinc                                                                        12
ferrous scrap specially processed grades meet consumer requirements cast iron grades         10
nonferrous scrap magnesium                                                                    5
ferrous scrap specially processed grades meet consumer requirements special boring grades     5
Name: sc:hasCategory, dtype: int64

In [16]:
df['sc:hasForm'].unique()

array(['wire', 'wire cable', 'miscellaneous wire',
       'clippings punchings bus bars commutator segments copper tubing',
       'miscellaneous scrap', 'wire scrap nodules',
       'valves machinery bearings machinery parts including miscellaneous castings',
       'scrap castings', 'scrap borings turnings', 'bushings bearings',
       'solids borings', 'turnings', 'solids', 'borings',
       'mixed solids castings tubings rolled rod plated',
       'castings crucible shape', 'clippings cuttings',
       'fired shell cases', 'rifle shells', 'primers', 'clippings',
       'clippings plate rods forgings rolled shapes',
       'sheet pipe rod tubes wire screen', 'pipe', 'castings',
       'borings turnings', 'rod ends forgings', 'rod turnings strictly',
       'rod ends', 'condenser tubes', 'tubes', 'lithographic sheets',
       'cans clippings', 'siding scrap', 'e c', 'extrusions', 'foil',
       'aluminum scrap', 'clippings solids',
       'castings forgings extrusions', 'wheels', 'ai

In [19]:
df['sc:contains'].unique()

array(['copper', 'red brass', 'brass', 'bronze', 'yellow brass', '',
       'nickel silver', 'leaded brass', 'aluminum brass', 'muntz metal',
       'manganese bronze', 'aluminum', 'zinc', 'magnesium',
       'wrought nickel', 'cupro nickel', 'copper nickel', 'r monel',
       'r monel k monel', 'k monel', 'monel', 'stainless steel',
       'wrougt iron steel', 'steel', 'black steel', 'iron steel',
       'series carbon steel', 'tin', 'terne', 'wrought iron steel',
       'cast iron malleable iron', 'iron', 'cast iron', 'burnt cast iron',
       'cast malleable iron', 'malleable iron'], dtype=object)

In [21]:
df['pc:hasName'].unique()

array(['copper wire', 'heavy copper solids tubing',
       'copper solids tubing', 'copper wire nodules',
       'insulated copper wire scrap', 'composition red brass',
       'lead free bismuth brass solids',
       'lead free bismuth brass turnings', '',
       'high lead bronze solids borings',
       'red brass composition turnings', 'machinery hard brass solids',
       'machinery hard brass borings', 'yellow brass scrap',
       'yellow brass castings', 'new brass clipings',
       'brass shell cases without primers',
       'brass small arms rifle shells clean fired',
       'brass small arms rifle shells clean muffled popped',
       'yellow brass primer', 'mixed new nickel silver clippings',
       'new nickel silver clippings solids',
       'new segregated nickel silver clippings', 'old nickel silver',
       'brass pipe', 'nickel silver castings',
       'leaded brass scrap turnings',
       'leaded brass scrap rod ends forgings',
       'yellow brass rod turnings', 'yellow

In [11]:
df.head(1)

,sc:hasCategory,sc:hasForm,sc:isFreeFrom,sc:contains,sc:hasSpecification,pc:hasName,sc:isComposedOf
0,nonferrous scrap red metals,wire,,copper,bare uncoated unalloyed,copper wire,sc:Material_249


In [22]:
# df['corpus'] = df['sc:hasCategory'] + ' ' + df['sc:hasSpecification'] + ' ' + df['sc:hasForm'] + df['sc:contains'] + ' ' + df['pc:hasName']
# df['corpus'] = df['sc:hasSpecification'] + ' ' + df['sc:hasForm'] + df['sc:contains']
df['corpus'] = df['sc:hasSpecification'] + ' ' + df['sc:hasForm'] + df['sc:contains'] + ' ' + df['pc:hasName']

In [23]:
df.head(2)

,sc:hasCategory,sc:hasForm,sc:isFreeFrom,sc:contains,sc:hasSpecification,pc:hasName,sc:isComposedOf,corpus
0,nonferrous scrap red metals,wire,,copper,bare uncoated unalloyed,copper wire,sc:Material_249,bare uncoated unalloyed wirecopper copper wire
1,nonferrous scrap red metals,wire cable,brittle burnt wire copper tubing,copper,clean untinned uncoated unalloyed,copper wire,sc:Material_249,clean untinned uncoated unalloyed wire cableco...


In [24]:
from sklearn.model_selection import train_test_split

X = df['corpus']
y = df['sc:isComposedOf']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

y_train = y_train.values
y_test = y_test.values

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

print(train_vectors.shape, test_vectors.shape)

(108, 521) (27, 521)


In [25]:
X.head()

0       bare uncoated unalloyed wirecopper copper wire
1    clean untinned uncoated unalloyed wire cableco...
2       unalloyed miscellaneous wirecopper copper wire
3    clean unalloyed uncoated clippings punchings b...
4    unalloyed miscellaneous scrapcopper copper sol...
Name: corpus, dtype: object

####  Logistic Regression

In [26]:
model = LogisticRegression()

model.fit(train_vectors, y_train)
predicted = model.predict(test_vectors)
print(accuracy_score(y_test,predicted))

results = pd.DataFrame(columns=['true', 'pred'])
results['true'] = y_test
results['pred'] = predicted

0.4074074074074074


#### SVC

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'kernel': ('linear', 'rbf') , 
              'C':[5, 10, 12, 15, 20, 25],
              'gamma': [1,0.1,0.2, 0.3, 0.5, 0.01,0.001]
             }
grid = GridSearchCV(SVC(random_state=42) , param_grid , refit=True , verbose=0, cv=5)
grid.fit(train_vectors, y_train)
print(grid.best_estimator_)

model_svc = grid.best_estimator_
predicted = model_svc.predict(test_vectors)
print(accuracy_score(y_test,predicted))

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


SVC(C=5, gamma=1, kernel='linear', random_state=42)
0.8518518518518519


#### KNN

In [28]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(train_vectors, y_train)
predicted = model.predict(test_vectors)
print(accuracy_score(y_test, predicted))

n_neighbors = list(range(1,20))
p=[1,2]#Convert to dictionary
param_grid = dict(n_neighbors=n_neighbors, p=p)

grid = GridSearchCV(model , param_grid , refit=True , verbose=0, cv=3)
grid.fit(train_vectors, y_train)
print(grid.best_estimator_)

model = grid.best_estimator_
predicted = model.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.8888888888888888


/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KNeighborsClassifier(n_neighbors=1)
0.8148148148148148


#### Random Forest Classifier

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest

model = RandomForestClassifier()

model.fit(train_vectors, y_train)
predicted = model.predict(test_vectors)
print(accuracy_score(y_test,predicted))

n_estimators = [int(x) for x in np.linspace(start = 3, stop = 100, num = 5)]
max_depth = [int(x) for x in np.linspace(2, 10, num = 1)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]# Create the random grid

param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

grid = RandomizedSearchCV(model , param_grid , refit=True , verbose=0, cv=3, n_iter=20)
grid.fit(train_vectors, y_train)
print(grid.best_estimator_)

model = grid.best_estimator_
predicted = model.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.7777777777777778


/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


RandomForestClassifier(bootstrap=False, n_estimators=51)
0.7777777777777778


#### Gaussian Process Classifier

In [31]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel

model = GaussianProcessClassifier()
model.fit(train_vectors.todense(), y_train)
predicted = model.predict(test_vectors.todense())
print(accuracy_score(y_test,predicted))

grid = dict()
grid['kernel'] = [1*RBF(), 1*DotProduct(), 1*Matern(),  1*RationalQuadratic(), 1*WhiteKernel()]
# define search
grid = GridSearchCV(model, grid, scoring='accuracy', cv=3, n_jobs=-1)
grid.fit(train_vectors.todense(), y_train)

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.2962962962962963


/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/v

GridSearchCV(cv=3, estimator=GaussianProcessClassifier(), n_jobs=-1,
             param_grid={'kernel': [1**2 * RBF(length_scale=1),
                                    1**2 * DotProduct(sigma_0=1),
                                    1**2 * Matern(length_scale=1, nu=1.5),
                                    1**2 * RationalQuadratic(alpha=1, length_scale=1),
                                    1**2 * WhiteKernel(noise_level=1)]},
             scoring='accuracy')

In [33]:
model_gp=grid.best_estimator_
model_gp

GaussianProcessClassifier(kernel=1**2 * DotProduct(sigma_0=1))

In [34]:
predicted = model_gp.predict(test_vectors.todense())
print(accuracy_score(y_test,predicted))

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.8888888888888888


#### Train best model on all `ISRI` dataset

In [35]:
vectors = vectorizer.transform(X)
print(vectors.shape)

(135, 521)


In [36]:
model_gp.fit(vectors.todense(), y)

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessClassifier(kernel=1**2 * DotProduct(sigma_0=1))

In [37]:
predicted = model_gp.predict(test_vectors.todense())
print(accuracy_score(y_test,predicted))

0.9629629629629629


/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [39]:
model_svc.fit(vectors, y)

SVC(C=5, gamma=1, kernel='linear', random_state=42)

The high accuracy is because we using the model to predict a part of the training set.

We will try to predict the material that `ANAMET` scraps are composed using the models trained on `ISRI` dataset

#### Keep only `ANAMET` with english `sc:description`

In [40]:
anamet_data = pd.read_excel(xls, 'Anamet data')

In [41]:
import unicodedata as ud

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha()) # isalpha suggested by John Machin

e_description_index = []
for row in range(len(anamet_data)):
    if only_roman_chars(str(anamet_data['sc:description'].iloc[row])):
        e_description_index.append(row)
        
anamet_data = anamet_data.iloc[e_description_index]
anamet_data.head(2)

,anamet entity,sc:description,sc:hasCategory,sc:internal_code,sc:isProcessedBy
28,sc:ANAMET_465,COPPER SCRAP MILLBERRY CABLES FOR STRIP.,Non-Ferrous,40014,NaN
30,sc:ANAMET_466,COPPER SCRAP BIRCH CABLES FOR STRIPPING,Non-Ferrous,40015,NaN


In [42]:
anamet_data.shape

(214, 5)

In [43]:
anamet_data = anamet_data[['anamet entity', 'sc:description', 'sc:hasCategory']]
anamet_data = anamet_data[['anamet entity', 'sc:description']]

anamet_data.head(2)

,anamet entity,sc:description
28,sc:ANAMET_465,COPPER SCRAP MILLBERRY CABLES FOR STRIP.
30,sc:ANAMET_466,COPPER SCRAP BIRCH CABLES FOR STRIPPING


In [44]:
anamet_data.isna().sum()

anamet entity     0
sc:description    0
dtype: int64

In [45]:
for col in anamet_data.drop('anamet entity', axis=1).columns:
    anamet_data[col] = hero.clean(anamet_data[col])

anamet_data.head(2)

,anamet entity,sc:description
28,sc:ANAMET_465,copper scrap millberry cables strip
30,sc:ANAMET_466,copper scrap birch cables stripping


In [46]:
anamet_data_X = pd.DataFrame()
# anamet_data_X['corpus'] = anamet_data['sc:description'] + ' ' + anamet_data['sc:hasCategory']
anamet_data_X['corpus'] = anamet_data['sc:description']

In [47]:
X = anamet_data_X['corpus']

anamet_train_vectors = vectorizer.transform(X)

print(anamet_train_vectors.shape)

(214, 521)


In [69]:
model = model_svc
y_hat = model.predict(anamet_train_vectors)

In [70]:
anamet_data['predicted material'] = y_hat

pd.set_option('display.max_rows', 300)
anamet_data

,anamet entity,sc:description,predicted material
28,sc:ANAMET_465,copper scrap millberry cables strip,sc:Material_249
30,sc:ANAMET_466,copper scrap birch cables stripping,sc:Material_249
33,sc:ANAMET_467,mixed copper cables scrap raw,sc:Material_249
34,sc:ANAMET_468,cables copper lead raw,sc:Material_249
37,sc:ANAMET_469,copper scrap berry candy per isr raw,sc:Material_249
38,sc:ANAMET_470,scrap ms raw,sc:Material_695
41,sc:ANAMET_471,brass scrap honey per isri raw,sc:Material_256
43,sc:ANAMET_472,aluminum scrap taint tabor raw material,sc:Material_244
45,sc:ANAMET_473,aluminum scrap mixed raw material,sc:Material_244
47,sc:ANAMET_474,aluminum scrap shred raw material,sc:Material_244


In [71]:
pd.DataFrame(y_hat).value_counts()

sc:Material_244    120
sc:Material_249     33
sc:Material_256     20
sc:Material_695     18
sc:Material_701     13
sc:Material_700      3
sc:Material_703      2
sc:Material_706      2
sc:Material_252      1
sc:Material_253      1
sc:Material_697      1
dtype: int64

In [73]:
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_697'] )
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_253'] )
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_252'] )

     anamet entity                sc:description predicted material
175  sc:ANAMET_542  scrap tlb pe without process    sc:Material_697
     anamet entity       sc:description predicted material
275  sc:ANAMET_642  metal gas tanks elv    sc:Material_253
     anamet entity               sc:description predicted material
140  sc:ANAMET_521  bronze scrap ebony per isri    sc:Material_252


In [75]:
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_700'] )
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_703'] )
print( anamet_data[anamet_data['predicted material'] == 'sc:Material_706'] )

     anamet entity                 sc:description predicted material
212  sc:ANAMET_579  electric motors shredded weee    sc:Material_700
260  sc:ANAMET_627       electric motors shredded    sc:Material_700
318  sc:ANAMET_685  electric motors shredded weee    sc:Material_700
     anamet entity      sc:description predicted material
261  sc:ANAMET_628  scrap complete car    sc:Material_703
266  sc:ANAMET_633  scrap car interior    sc:Material_703
     anamet entity       sc:description predicted material
102  sc:ANAMET_502  copper nickel scrap    sc:Material_706
104  sc:ANAMET_503  copper nickel scrap    sc:Material_706


#### Gaussian Process Model

In [66]:
model = model_gp
y_hat = model.predict(anamet_train_vectors.todense())

/home/comex/Desktop/python-envs/main-bio/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [67]:
anamet_data['predicted material'] = y_hat

pd.set_option('display.max_rows', 300)
anamet_data

,anamet entity,sc:description,predicted material
28,sc:ANAMET_465,copper scrap millberry cables strip,sc:Material_249
30,sc:ANAMET_466,copper scrap birch cables stripping,sc:Material_249
33,sc:ANAMET_467,mixed copper cables scrap raw,sc:Material_249
34,sc:ANAMET_468,cables copper lead raw,sc:Material_249
37,sc:ANAMET_469,copper scrap berry candy per isr raw,sc:Material_249
38,sc:ANAMET_470,scrap ms raw,sc:Material_695
41,sc:ANAMET_471,brass scrap honey per isri raw,sc:Material_256
43,sc:ANAMET_472,aluminum scrap taint tabor raw material,sc:Material_244
45,sc:ANAMET_473,aluminum scrap mixed raw material,sc:Material_244
47,sc:ANAMET_474,aluminum scrap shred raw material,sc:Material_244


In [68]:
pd.DataFrame(y_hat).value_counts()

sc:Material_244    99
sc:Material_249    37
sc:Material_695    31
sc:Material_256    20
sc:Material_701    13
sc:Material_700     3
sc:Material_703     3
sc:Material_697     2
sc:Material_704     2
sc:Material_247     1
sc:Material_252     1
sc:Material_253     1
sc:Material_696     1
dtype: int64